In [1]:
#Import all the packages needed 
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
from scipy.interpolate import interp1d
#import the package to generate a random number, note that seed is use to set a series of random number that can be reused
#ie for rooftop solar and utility solar to use the same sequenece of random number
#because if the sun is shinny, and if we have good rooftop solar production, utility solar production should also be good 
from random import random, seed 
#this is the package needed to generate an empirical distribution function
from statsmodels.distributions.empirical_distribution import ECDF

#pd.set_option('display.max_rows', None)


In [2]:
#here we set the numbers of scenario that will be generated for winter, summer, and other (autumn and spring) 
#nfor example, the number 247 is obtained by deviding 90 days of winter by 365 days in a year; multiply by 1000 scenario
#this ensure the stochastic scenarios we generated follow the proportion of seasonal distribution of supply/demand
winter_probabilities = 247
summer_probabilities = 252
other_probabilities  = 501

In [3]:
#this section set the if statement use to classify each day of raw data obtained from renewable.ninja (NASA's data) into seasons

def season_setting(row):
    day = row.local_time.dayofyear
    spring = range(80, 172)
    summer = range(172, 264)
    fall = range(264, 355)
    
    if day in spring:
        season = 'other'
    elif day in summer:
        season = 'summer'
    elif day in fall:
        season = 'other'
    else:
        season = 'winter'
    return season

In [24]:
#seed 1, is the sequence of random number used for all solar random number generation
seed(1)

pathName = os.getcwd()
fileNames = os.listdir(pathName)
numFiles = []
#this is to import/read all 29 files with the ending RS.csv; for each node
for fileNames in fileNames:
    if fileNames.endswith("RS.csv"):
        numFiles.append(fileNames)
print(numFiles)

#a stochastic number is generated for each node, at each hour, of each scenario
hours = range(0,24)

RS_CF = pd.DataFrame()

for f, file in enumerate(numFiles):
    #this is just to clean the raw file gotten from renewable ninja; and only consider the column we care about

    file_name = 'Node ' + str(f)
    
    t = pd.read_csv(file, skiprows = 3, usecols = ['local_time','electricity'],\
                    parse_dates = ['local_time'],\
                    infer_datetime_format = True, dayfirst = True)
    #assigning a season to each day (each with 24 hours) within the raw data obtained
    t['season'] = t.apply(season_setting, axis = 1)
    winter_days = t[t['season'] == 'winter']
    summer_days = t[t['season'] == 'summer']
    other_days  = t[t['season'] == 'other']
    seasons = [winter_days, summer_days, other_days]


    for n, season in enumerate(seasons):
        if(n == 0):
            season_name = 'winter'
        elif(n == 1):
            season_name = 'summer'
        else:
            season_name = 'other'
            
            #create a random number for each hour, at each node, within a scenario
        for hour in hours:
            winter_random_probabilities = [random() for _ in range(winter_probabilities)]
            summer_random_probabilities = [random() for _ in range(summer_probabilities)]
            other_random_probabilities  = [random() for _ in range(other_probabilities) ]
            season_probabilities = [winter_random_probabilities, summer_random_probabilities, other_random_probabilities]
            
            #creating temperory data frame for each output, so that it can all be put together 
            temp_df = pd.DataFrame()
            #use the data from the 'electricity' col in each raw data file (how much electricity is generated for a 1kW plant)
            #to create an empirical cummulative distribution function - for each location in that season, at that hour
            hourly_data = season[season.local_time.dt.time == dt.time(hour)]['electricity']
            ecdf = ECDF(hourly_data)
            x = np.linspace(min(hourly_data), max(hourly_data), 1000)
            slope_changes = sorted(set(x))
            #0 slope change means all values are 0 for solar irradiance (because it is night time)
            #temp_df = pd.DataFrame()
            if(len(slope_changes) == 1):
                for k, random_value in enumerate(season_probabilities[n]):
                    capacity_factor = 0
                    row = {'RS CF %s Hour %s Season %s' %(file_name, hour, season_name) : capacity_factor}
                    temp_df = temp_df.append(row, ignore_index = True)
                    
            #create a random number (between 0 and 1); this number is ten correspnded to the probability value that
            #is plugged in to find the corresponding value of solar energy production at that instance; creating
            #stochastic number of solar energy production at that hour that internalize the distribution of solar raw data
            else:
                temp_df = pd.DataFrame()
                sample_edf_values_at_slope_changes = [ ecdf(item) for item in slope_changes]
                inverted_edf = interp1d(sample_edf_values_at_slope_changes, slope_changes, fill_value='extrapolate')
                for k, random_value in enumerate(season_probabilities[n]):
                    capacity_factor = float(inverted_edf(random_value))
                    if(capacity_factor < 0):
                        capacity_factor = 0
                    row = {'RS CF %s Hour %s Season %s' %(file_name, hour, season_name) : capacity_factor}
                    temp_df = temp_df.append(row, ignore_index = True)
            
            RS_CF = pd.concat([RS_CF,temp_df], axis = 1).sort_index()
RS_CF

['Node1RS.csv', 'Node2RS.csv', 'Node3RS.csv', 'Node4RS.csv', 'Node5RS.csv', 'Node6RS.csv', 'Node7RS.csv', 'Node8RS.csv', 'Node9RS.csv', 'Nodf10RS.csv', 'Nodf11RS.csv', 'Nodf12RS.csv', 'Nodf13RS.csv', 'Nodf14RS.csv', 'Nodf15RS.csv', 'Nodf16RS.csv', 'Nodf17RS.csv', 'Nodf18RS.csv', 'Nodf19RS.csv', 'Nodf20RS.csv', 'Nodf21RS.csv', 'Nodf22RS.csv', 'Nodf23RS.csv', 'Nodf24RS.csv', 'Nodf25RS.csv', 'Nodf26RS.csv', 'Nodf27RS.csv', 'Nodf28RS.csv', 'Nodf29RS.csv']


C:\Users\Bader\anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


,RS CF Node 0 Hour 0 Season winter,RS CF Node 0 Hour 1 Season winter,RS CF Node 0 Hour 2 Season winter,RS CF Node 0 Hour 3 Season winter,RS CF Node 0 Hour 4 Season winter,RS CF Node 0 Hour 5 Season winter,RS CF Node 0 Hour 6 Season winter,RS CF Node 0 Hour 7 Season winter,RS CF Node 0 Hour 8 Season winter,RS CF Node 0 Hour 9 Season winter,...,RS CF Node 28 Hour 14 Season other,RS CF Node 28 Hour 15 Season other,RS CF Node 28 Hour 16 Season other,RS CF Node 28 Hour 17 Season other,RS CF Node 28 Hour 18 Season other,RS CF Node 28 Hour 19 Season other,RS CF Node 28 Hour 20 Season other,RS CF Node 28 Hour 21 Season other,RS CF Node 28 Hour 22 Season other,RS CF Node 28 Hour 23 Season other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.073652,0.217791,...,0.719790,0.123363,0.516342,0.088967,0.000000,0.009129,0.017671,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.088070,0.014859,...,0.606781,0.004906,0.509253,0.167479,0.018654,0.000000,0.005476,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000969,...,0.122892,0.019505,0.287244,0.017011,0.077005,0.052272,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008827,0.025026,0.108839,...,0.487493,0.273639,0.128940,0.036605,0.035875,0.000000,0.001127,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000937,...,0.334290,0.627689,0.283848,0.127524,0.008426,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.693121,0.176286,0.402352,0.073060,0.000000,0.035549,0.007261,0.0,0.0,0.0
497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.520918,0.125837,0.000000,0.000000,0.213254,0.000000,0.000000,0.0,0.0,0.0
498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.442443,0.018009,0.000000,0.302020,0.000000,0.000000,0.004060,0.0,0.0,0.0
499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.122863,0.636850,0.000000,0.338249,0.016448,0.000000,0.016365,0.0,0.0,0.0


In [25]:
RS_CF.to_excel('Rooftop Solar Stochastic Output.xlsx')

In [26]:
seed(1)

US_CF = pd.DataFrame()

pathName = os.getcwd()
fileNames = os.listdir(pathName)
numFiles = []
for fileNames in fileNames:
    if fileNames.endswith("US.csv"):
        numFiles.append(fileNames)
print(numFiles)

for f, file in enumerate(numFiles):
    
    file_name = 'Node ' + str(f)
    
    t = pd.read_csv(file, skiprows = 3, usecols = ['local_time','electricity'],\
                    parse_dates = ['local_time'],\
                    infer_datetime_format = True, dayfirst = True)
    
    t['season'] = t.apply(season_setting, axis = 1)
    winter_days = t[t['season'] == 'winter']
    summer_days = t[t['season'] == 'summer']
    other_days  = t[t['season'] == 'other']
    seasons = [winter_days, summer_days, other_days]

    for n, season in enumerate(seasons):
        if(n == 0):
            season_name = 'winter'
        elif(n == 1):
            season_name = 'summer'
        else:
            season_name = 'other'
        
        for hour in hours:
            winter_random_probabilities = [random() for _ in range(winter_probabilities)]
            summer_random_probabilities = [random() for _ in range(summer_probabilities)]
            other_random_probabilities  = [random() for _ in range(other_probabilities) ]
            season_probabilities = [winter_random_probabilities, summer_random_probabilities, other_random_probabilities]
            
          
            hourly_data = season[season.local_time.dt.time == dt.time(hour)]['electricity']
            ecdf = ECDF(hourly_data)
            x = np.linspace(min(hourly_data), max(hourly_data), 1000)
            slope_changes = sorted(set(x))
            #0 slope change means all values are 0 for solar irradiance (because it is night time)
            temp_df = pd.DataFrame()
            if(len(slope_changes) == 1):
                for k, random_value in enumerate(season_probabilities[n]):
                    capacity_factor = 0
                    row = {'US CF %s Hour %s Season %s' %(file_name, hour, season_name) : capacity_factor}
                    temp_df = temp_df.append(row, ignore_index = True)
            
            else:
                sample_edf_values_at_slope_changes = [ ecdf(item) for item in slope_changes]
                inverted_edf = interp1d(sample_edf_values_at_slope_changes, slope_changes, fill_value='extrapolate')
                for k, random_value in enumerate(season_probabilities[n]):
                    capacity_factor = float(inverted_edf(random_value))
                    if(capacity_factor < 0):
                        capacity_factor = 0
                    row = {'US CF %s Hour %s Season %s' %(file_name, hour, season_name) : capacity_factor}
                    temp_df = temp_df.append(row, ignore_index = True)
            US_CF = pd.concat([US_CF,temp_df], axis = 1).sort_index()
US_CF

['Node1US.csv', 'Node2US.csv', 'Node3US.csv', 'Node4US.csv', 'Node5US.csv', 'Node6US.csv', 'Node7US.csv', 'Node8US.csv', 'Node9US.csv', 'Nodf10US.csv', 'Nodf11US.csv', 'Nodf12US.csv', 'Nodf13US.csv', 'Nodf14US.csv', 'Nodf15US.csv', 'Nodf16US.csv', 'Nodf17US.csv', 'Nodf18US.csv', 'Nodf19US.csv', 'Nodf20US.csv', 'Nodf21US.csv', 'Nodf22US.csv', 'Nodf23US.csv', 'Nodf24US.csv', 'Nodf25US.csv', 'Nodf26US.csv', 'Nodf27US.csv', 'Nodf28US.csv', 'Nodf29US.csv']


C:\Users\Bader\anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


,US CF Node 0 Hour 0 Season winter,US CF Node 0 Hour 1 Season winter,US CF Node 0 Hour 2 Season winter,US CF Node 0 Hour 3 Season winter,US CF Node 0 Hour 4 Season winter,US CF Node 0 Hour 5 Season winter,US CF Node 0 Hour 6 Season winter,US CF Node 0 Hour 7 Season winter,US CF Node 0 Hour 8 Season winter,US CF Node 0 Hour 9 Season winter,...,US CF Node 28 Hour 14 Season other,US CF Node 28 Hour 15 Season other,US CF Node 28 Hour 16 Season other,US CF Node 28 Hour 17 Season other,US CF Node 28 Hour 18 Season other,US CF Node 28 Hour 19 Season other,US CF Node 28 Hour 20 Season other,US CF Node 28 Hour 21 Season other,US CF Node 28 Hour 22 Season other,US CF Node 28 Hour 23 Season other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.092262,0.275759,...,0.756094,0.126633,0.706433,0.118053,0.000000,0.018976,0.078021,0.0000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.114510,0.017041,...,0.661265,0.004272,0.685640,0.258380,0.031271,0.000000,0.027512,0.0000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.125776,0.019721,0.406064,0.014390,0.106478,0.168431,0.000000,0.0000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010911,0.027682,0.121897,...,0.538354,0.315322,0.127312,0.034939,0.046319,0.000000,0.002997,0.0000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.366631,0.729198,0.380828,0.179813,0.013501,0.000000,0.000000,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.741561,0.217217,0.569810,0.085364,0.000000,0.098051,0.033407,0.0027,0.0,0.0
497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.585239,0.153250,0.000000,0.000000,0.593233,0.000000,0.000000,0.0000,0.0,0.0
498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.493309,0.014826,0.000000,0.514122,0.000000,0.000000,0.017092,0.0000,0.0,0.0
499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.125725,0.730689,0.000000,0.608019,0.025085,0.000000,0.068877,0.0000,0.0,0.0


In [27]:
US_CF.to_excel('Utility Scale Solar Stochastic Output.xlsx')

In [32]:
seed(2)

ON_CF = pd.DataFrame()

pathName = os.getcwd()
fileNames = os.listdir(pathName)
numFiles = []
for fileNames in fileNames:
    if fileNames.endswith("ON.csv"):
        numFiles.append(fileNames)
print(numFiles)

for f, file in enumerate(numFiles):
    
    file_name = 'Node ' + str(f)
    print('File ' + file_name + 'started')
    t = pd.read_csv(file, skiprows = 3, usecols = ['local_time','electricity'],\
                    parse_dates = ['local_time'],\
                    infer_datetime_format = True, dayfirst = True)
    
    t['season'] = t.apply(season_setting, axis = 1)
    winter_days = t[t['season'] == 'winter']
    summer_days = t[t['season'] == 'summer']
    other_days  = t[t['season'] == 'other']
    seasons = [winter_days, summer_days, other_days]
    

    for n, season in enumerate(seasons):
        if(n == 0):
            season_name = 'winter'
        elif(n == 1):
            season_name = 'summer'
        else:
            season_name = 'other'
        
        for hour in hours:
            
            winter_random_probabilities = [random() for _ in range(winter_probabilities)]
            summer_random_probabilities = [random() for _ in range(summer_probabilities)]
            other_random_probabilities  = [random() for _ in range(other_probabilities) ]
            season_probabilities = [winter_random_probabilities, summer_random_probabilities, other_random_probabilities]
            
            hourly_data = season[season.local_time.dt.time == dt.time(hour)]['electricity']
            try:
                ecdf = ECDF(hourly_data)
                x = np.linspace(min(hourly_data), max(hourly_data), 1000)
                slope_changes = sorted(set(x))
                #0 slope change means all values are 0 for solar irradiance (because it is night time)
                temp_df = pd.DataFrame()
                if(len(slope_changes) == 1):
                    for k, random_value in enumerate(season_probabilities[n]):
                        capacity_factor = 0
                        row = {'ON CF %s Hour %s Season %s' %(file_name, hour, season_name) : capacity_factor}
                        temp_df = temp_df.append(row, ignore_index = True)
                
                else:
                    sample_edf_values_at_slope_changes = [ ecdf(item) for item in slope_changes]
                    inverted_edf = interp1d(sample_edf_values_at_slope_changes, slope_changes, fill_value='extrapolate')
                    for k, random_value in enumerate(season_probabilities[n]):
                        capacity_factor = float(inverted_edf(random_value))
                        if(capacity_factor < 0):
                            capacity_factor = 0
                        row = {'ON CF %s Hour %s Season %s' %(file_name, hour, season_name) : capacity_factor}
                        temp_df = temp_df.append(row, ignore_index = True)
                ON_CF = pd.concat([ON_CF,temp_df], axis = 1).sort_index()
            except:
                print(hourly_data)
ON_CF

['Node1ON.csv', 'Node2ON.csv', 'Node3ON.csv', 'Node4ON.csv', 'Node5ON.csv', 'Node6ON.csv', 'Node7ON.csv', 'Node8ON.csv', 'Node9ON.csv', 'Nodf10ON.csv', 'Nodf11ON.csv', 'Nodf12ON.csv', 'Nodf13ON.csv', 'Nodf14ON.csv', 'Nodf15ON.csv', 'Nodf16ON.csv', 'Nodf17ON.csv', 'Nodf18ON.csv', 'Nodf19ON.csv', 'Nodf20ON.csv', 'Nodf21ON.csv', 'Nodf22ON.csv', 'Nodf23ON.csv', 'Nodf24ON.csv', 'Nodf25ON.csv', 'Nodf26ON.csv', 'Nodf27ON.csv', 'Nodf28ON.csv', 'Nodf29ON.csv']
File Node 0started


C:\Users\Bader\anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


File Node 1started
File Node 2started
File Node 3started
File Node 4started
File Node 5started
File Node 6started
File Node 7started
File Node 8started
File Node 9started
File Node 10started
File Node 11started
File Node 12started
File Node 13started
File Node 14started
File Node 15started
File Node 16started
File Node 17started
File Node 18started
File Node 19started
File Node 20started
File Node 21started
File Node 22started
File Node 23started
File Node 24started
File Node 25started
File Node 26started
File Node 27started
File Node 28started


,ON CF Node 0 Hour 0 Season winter,ON CF Node 0 Hour 1 Season winter,ON CF Node 0 Hour 2 Season winter,ON CF Node 0 Hour 3 Season winter,ON CF Node 0 Hour 4 Season winter,ON CF Node 0 Hour 5 Season winter,ON CF Node 0 Hour 6 Season winter,ON CF Node 0 Hour 7 Season winter,ON CF Node 0 Hour 8 Season winter,ON CF Node 0 Hour 9 Season winter,...,ON CF Node 28 Hour 14 Season other,ON CF Node 28 Hour 15 Season other,ON CF Node 28 Hour 16 Season other,ON CF Node 28 Hour 17 Season other,ON CF Node 28 Hour 18 Season other,ON CF Node 28 Hour 19 Season other,ON CF Node 28 Hour 20 Season other,ON CF Node 28 Hour 21 Season other,ON CF Node 28 Hour 22 Season other,ON CF Node 28 Hour 23 Season other
0,0.942158,0.953807,0.219445,0.670462,0.677500,0.046725,0.577091,0.403600,0.969043,0.239386,...,0.482042,0.749470,0.262970,0.680303,0.809741,0.025061,0.214891,0.352967,0.614016,0.575361
1,0.917970,0.926875,0.102673,0.504439,0.941547,0.055695,0.526795,0.183141,0.213589,0.960709,...,0.051148,0.432482,0.486801,0.530091,0.029921,0.633295,0.041702,0.275967,0.333789,0.496333
2,0.044048,0.584596,0.947531,0.230289,0.063160,0.707528,0.348199,0.745180,0.709199,0.031165,...,0.090719,0.277213,0.125346,0.000000,0.625735,0.489962,0.454808,0.190046,0.481401,0.686780
3,0.088548,0.276177,0.450739,0.583609,0.197876,0.038719,0.442014,0.768449,0.419343,0.265048,...,0.382910,0.835880,0.400027,0.144284,0.941555,0.090057,0.208037,0.361152,0.026587,0.785013
4,0.785777,0.928449,0.850747,0.202149,0.668610,0.130115,0.601609,0.069598,0.317727,0.792652,...,0.528066,0.404405,0.482978,0.762441,0.537114,0.733543,0.338812,0.146423,0.000000,0.920527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.097772,0.925673,0.600961,0.265498,0.619702,0.152922,0.566832,0.960712,0.666794,0.020161
497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.144408,0.616655,0.848318,0.048815,0.034995,0.204830,0.107880,0.569873,0.369646,0.037017
498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.308047,0.226337,0.043913,0.054521,0.271371,0.366446,0.369636,0.581608,0.831017,0.151957
499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.823287,0.079842,0.247378,0.429202,0.336775,0.647217,0.867683,0.104721,0.000000,0.044708


In [33]:
ON_CF.to_excel('Onshore Wind Stochastic Output New N2.xlsx')

In [34]:
seed(3)

pathName = os.getcwd()
fileNames = os.listdir(pathName)
numFiles = []
for fileNames in fileNames:
    if fileNames.endswith("OF.csv"):
        numFiles.append(fileNames)
print(numFiles)

hours = range(0,24)

OF_CF = pd.DataFrame()

for f, file in enumerate(numFiles):

    file_name = 'Node ' + str(f)
    
    t = pd.read_csv(file, skiprows = 3, usecols = ['local_time','electricity'],\
                    parse_dates = ['local_time'],\
                    infer_datetime_format = True, dayfirst = True)
    
    t['season'] = t.apply(season_setting, axis = 1)
    winter_days = t[t['season'] == 'winter']
    summer_days = t[t['season'] == 'summer']
    other_days  = t[t['season'] == 'other']
    seasons = [winter_days, summer_days, other_days]
    


    for n, season in enumerate(seasons):
        if(n == 0):
            season_name = 'winter'
        elif(n == 1):
            season_name = 'summer'
        else:
            season_name = 'other'
        for hour in hours:
            winter_random_probabilities = [random() for _ in range(winter_probabilities)]
            summer_random_probabilities = [random() for _ in range(summer_probabilities)]
            other_random_probabilities  = [random() for _ in range(other_probabilities) ]
            season_probabilities = [winter_random_probabilities, summer_random_probabilities, other_random_probabilities]
            
            temp_df = pd.DataFrame()
            hourly_data = season[season.local_time.dt.time == dt.time(hour)]['electricity']
            ecdf = ECDF(hourly_data)
            x = np.linspace(min(hourly_data), max(hourly_data), 1000)
            slope_changes = sorted(set(x))
            #0 slope change means all values are 0 for solar irradiance (because it is night time)
            #temp_df = pd.DataFrame()
            if(len(slope_changes) == 1):
                for k, random_value in enumerate(season_probabilities[n]):
                    capacity_factor = 0
                    row = {'OF CF %s Hour %s Season %s' %(file_name, hour, season_name) : capacity_factor}
                    temp_df = temp_df.append(row, ignore_index = True)
            
            else:
                temp_df = pd.DataFrame()
                sample_edf_values_at_slope_changes = [ ecdf(item) for item in slope_changes]
                inverted_edf = interp1d(sample_edf_values_at_slope_changes, slope_changes, fill_value='extrapolate')
                for k, random_value in enumerate(season_probabilities[n]):
                    capacity_factor = float(inverted_edf(random_value))
                    if(capacity_factor < 0):
                        capacity_factor = 0
                    row = {'OF CF %s Hour %s Season %s' %(file_name, hour, season_name) : capacity_factor}
                    temp_df = temp_df.append(row, ignore_index = True)
            
            OF_CF = pd.concat([OF_CF,temp_df], axis = 1).sort_index()
OF_CF

['Node1OF.csv', 'Node3OF.csv', 'Node4OF.csv', 'Node7OF.csv', 'Node9OF.csv', 'Nodf11OF.csv', 'Nodf12OF.csv', 'Nodf16OF.csv', 'Nodf19OF.csv', 'Nodf20OF.csv', 'Nodf26OF.csv', 'Nodf27OF.csv', 'Nodf28OF.csv', 'Nodf29OF.csv']


C:\Users\Bader\anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


,OF CF Node 0 Hour 0 Season winter,OF CF Node 0 Hour 1 Season winter,OF CF Node 0 Hour 2 Season winter,OF CF Node 0 Hour 3 Season winter,OF CF Node 0 Hour 4 Season winter,OF CF Node 0 Hour 5 Season winter,OF CF Node 0 Hour 6 Season winter,OF CF Node 0 Hour 7 Season winter,OF CF Node 0 Hour 8 Season winter,OF CF Node 0 Hour 9 Season winter,...,OF CF Node 13 Hour 14 Season other,OF CF Node 13 Hour 15 Season other,OF CF Node 13 Hour 16 Season other,OF CF Node 13 Hour 17 Season other,OF CF Node 13 Hour 18 Season other,OF CF Node 13 Hour 19 Season other,OF CF Node 13 Hour 20 Season other,OF CF Node 13 Hour 21 Season other,OF CF Node 13 Hour 22 Season other,OF CF Node 13 Hour 23 Season other
0,0.434761,0.372921,0.110787,0.351450,0.442956,0.983412,0.895514,0.925003,0.976194,0.989997,...,0.802042,0.071988,0.892225,0.837177,0.115227,0.910662,0.050416,0.555118,0.626603,0.067570
1,0.778270,0.134417,0.460644,0.096418,0.934918,0.750724,0.163367,0.778885,0.919400,0.097272,...,0.800951,0.873640,0.985771,0.511171,0.168113,0.292148,0.906823,0.025381,0.684810,0.137077
2,0.612786,0.990509,0.875488,0.694569,0.329587,0.928006,0.784402,0.990326,0.981995,0.119365,...,0.072191,0.163112,0.089528,0.115726,0.038195,0.383722,0.724344,0.461358,0.212796,0.108148
3,0.808857,0.618407,0.991523,0.745254,0.989436,0.802387,0.989956,0.085206,0.797083,0.764814,...,0.495515,0.518884,0.913631,0.038117,0.431004,0.299932,0.989198,0.031358,0.842358,0.048232
4,0.813195,0.974787,0.837524,0.913689,0.231408,0.983644,0.989158,0.902602,0.504866,0.976920,...,0.293937,0.725319,0.818674,0.727256,0.288647,0.729397,0.400106,0.527036,0.058721,0.194155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.076881,0.799268,0.543925,0.787604,0.505910,0.066607,0.690287,0.838190,0.072024,0.957100
497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.077792,0.398202,0.086743,0.097237,0.490452,0.456508,0.624083,0.592827,0.510355,0.539684
498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.104606,0.854538,0.895772,0.189116,0.477040,0.230026,0.303579,0.039211,0.144405,0.629692
499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.265204,0.460399,0.733813,0.026203,0.323733,0.500289,0.101731,0.670238,0.122124,0.704820


In [35]:
OF_CF.to_excel('Offshore Wind Stochastic Output new sto seed.xlsx')